In [ ]:
!pip install pyngrok flask


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving app.py to app.py
Saving poultry_disease_model.h5 to poultry_disease_model.h5


In [ ]:
from flask import Flask, request, render_template_string
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from pyngrok import ngrok
import numpy as np
import os

# Setup
app = Flask(__name__)
model = load_model("poultry_disease_model.h5")

class_names = ['Coccidiosis', 'Healthy', 'New Castle Disease', 'Salmonella']
disease_info = {
    'Coccidiosis': "🦠 A parasitic disease affecting intestines. Treat with Amprolium or sulfa drugs.",
    'Healthy': "✅ The bird is healthy. Maintain hygiene and vaccination schedule.",
    'New Castle Disease': "😷 A contagious viral disease. Vaccinate and isolate affected birds.",
    'Salmonella': "🧬 Bacterial infection. Use antibiotics like tetracycline. Ensure feed hygiene."
}

# HTML Template
HTML_TEMPLATE = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Poultry Disease Classifier</title>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <!-- Bootstrap CSS CDN -->
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body class="bg-light">

<div class="container mt-5">
    <div class="card shadow-sm p-4">
        <h2 class="text-center mb-4">🐔 Poultry Disease Prediction</h2>
        <form method="POST" enctype="multipart/form-data" class="text-center">
            <div class="mb-3">
                <input class="form-control" type="file" name="image" required>
            </div>
            <button class="btn btn-primary" type="submit">🔍 Predict</button>
        </form>

        {% if prediction %}
        <div class="mt-4 text-center">
            <h4 class="text-success">{{ prediction }}</h4>
            <p class="text-muted">{{ info }}</p>
            <img src="{{ img_path }}" class="img-fluid rounded shadow" width="300">
        </div>
        {% endif %}
    </div>
</div>

</body>
</html>
'''

# Create uploads folder
os.makedirs("uploads", exist_ok=True)

@app.route("/", methods=["GET", "POST"])
def index():
    prediction = None
    info = None

    if request.method == "POST":
        file = request.files["image"]
        if file:
            path = os.path.join("uploads", file.filename)
            file.save(path)

            img = image.load_img(path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0) / 255.0

            pred = model.predict(img_array)
            class_index = np.argmax(pred)
            confidence = float(pred[0][class_index]) * 100
            prediction = f"{class_names[class_index]} ({confidence:.2f}% confidence)"
            info = disease_info[class_names[class_index]]


    return render_template_string(HTML_TEMPLATE, prediction=prediction, info=info)
from pyngrok import ngrok

# Paste YOUR token here:
ngrok.set_auth_token("2ypP1k2AZrxATrpEOyJT1Zs1Euu_6cHmHo3eQNPsH4Dd9mR5B")
# Start the Flask app using pyngrok
public_url = ngrok.connect(5000)
print("🌐 Public URL:", public_url)

app.run(port=5000)


🌐 Public URL: NgrokTunnel: "https://147f-34-106-68-213.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/Jun/2025 18:03:52] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jun/2025 18:03:57] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


INFO:werkzeug:127.0.0.1 - - [21/Jun/2025 18:04:10] "POST / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step


INFO:werkzeug:127.0.0.1 - - [21/Jun/2025 18:04:29] "POST / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step


INFO:werkzeug:127.0.0.1 - - [21/Jun/2025 18:04:39] "POST / HTTP/1.1" 200 -
